In [1]:
import json
import os
import sys
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

sys.path.append('..')
from soysaucestock.stock_price_scraper import StockChartScraper
from soysaucestock.stock_chart_backtester import StockChartTester, TradeResult
from soysaucestock.technical_analyzer import TechnicalAnalyzer

In [2]:
class ScanResult(object):
    def __init__(self, date_str, ticker, close, volume):
        self.date_str = date_str
        self.ticker = ticker
        self.close = close
        self.volume = volume
    def tostring(self):
        return str.format("{0},{1}", self.date_str, self.ticker)

In [3]:
def get_scan_result(driver, date_pd, date_range):
    driver.implicitly_wait(2)
    result = []
    for date_index in date_range:
        driver.find_element_by_id('globalDateOffset').send_keys(Keys.CONTROL + 'a' + Keys.NULL, str(date_index))
        parent_handle = driver.window_handles[0]
        driver.find_element_by_css_selector('input[value="Run Scan"]').click()
        result_handle = driver.window_handles[1]
        driver.switch_to_window(result_handle)
        trs = driver.find_elements_by_xpath("//table[@id='scc-scans-resultstable']/tbody/tr")
        for result_index in range(0, len(trs)):
            tr = trs[result_index]
            tr_list = tr.text.split(' ')
            scan_result = ScanResult(date_str=date_pd.index[date_index+1], ticker=tr_list[0], close=tr_list[-2], volume=tr_list[-1])
            result.append(scan_result)
        driver.switch_to_window(parent_handle)
    driver.implicitly_wait(10)
    return result

In [4]:
StockChartScraper.save_stocks('RY.TO', is_lazy_save=False)

In [5]:
driver = webdriver.Firefox()
driver.implicitly_wait(10) # seconds
driver.get("http://stockcharts.com")

In [6]:
account_json = 'C:\\Tools\\stockchart.json'
with open(account_json) as data_file:    
    account_info = json.load(data_file)
driver.find_element_by_link_text('Login').click()
driver.find_element_by_id('form_UserID').send_keys(account_info['user_name'])
driver.find_element_by_name('form_UserPassword').send_keys(account_info['password'])
driver.find_element_by_id('loginbutton').submit()

In [7]:
driver.find_element_by_name('button_GoScanDef').click()
select = Select(driver.find_element_by_id('favScans'))
select.select_by_visible_text('Daily Bear Down')


In [8]:
date_mapping = 'C:\\temp\\RY.TO.csv'
date_pd = pd.read_csv(date_mapping, parse_dates=False, index_col=0)


In [9]:
date_pd = date_pd.sort_index(ascending=False)

In [10]:
result = get_scan_result(driver, date_pd, range(0, 200))

In [11]:
target_path = 'C:\\temp\\ForceIndexBear.csv'

with open(target_path, 'w') as f:
    f.write('Date,Ticker\n')
    for item in result:
        f.write(item.tostring() +'\n')

In [12]:
select = Select(driver.find_element_by_id('favScans'))
select.select_by_visible_text('Daily Bull Up')

In [13]:
result = get_scan_result(driver, date_pd, range(0, 200))

In [14]:
target_path = 'C:\\temp\\ForceIndexBull.csv'
with open(target_path, 'w') as f:
    f.write('Date,Ticker\n')
    for item in result:
        f.write(item.tostring() +'\n')